In [1]:
from keras.models import load_model as keras_load_model
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from keras.models import load_model
import tensorflow as tf
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from prophet import Prophet
import pandas as pd
import numpy as np
import warnings
import pickle
import joblib
import os
warnings.filterwarnings('ignore')


c:\Anaconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 현재 날짜와 시간 가져오기
today = datetime.today()

# 날짜만 가져오기
current_date = today.date()

print("오늘 날짜:", current_date)

df = pd.read_excel('../../data/tb_stock.xlsx')

오늘 날짜: 2024-09-30


## SARIMA

In [3]:
def sarima_predict_up_to_date(stock, model_path, input_date_str=current_date, data_path='../../data/tb_stock.xlsx',  lookback_days=60, steps_ahead=14):
    """
    지정된 날짜까지 예측을 수행하는 함수.

    :param input_date_str   : 사용자가 입력한 날짜 (YYYY-MM-DD 형식의 문자열)
    :param data_path        : 주식 데이터가 저장된 엑셀 파일 경로
    :param model_path       : 저장된 SARIMA 모델 경로
    :param lookback_days    : 예측에 사용할 최근 데이터의 일 수 (기본값: 60일)
    :param steps_ahead      : 한 번에 예측할 일 수 (기본값: 14일)
    :return                 : 예측된 데이터 프레임
    """

    # 입력된 날짜를 날짜 형식으로 변환
    input_date = pd.to_datetime(input_date_str)

    # 엑셀 데이터 불러오기
    df = pd.read_excel(data_path)
    df['sc_date'] = pd.to_datetime(df['sc_date'])
    df.set_index('sc_date', inplace=True)
    df = df.asfreq('D').fillna(method='ffill')  # 결측치를 전날 데이터로 채움

    # 모델 불러오기
    with open(model_path, 'rb') as pkl_file:
        loaded_model = pickle.load(pkl_file)

    # 입력된 날짜 이전의 데이터만 사용
    df_filtered = df[df.index < input_date]

    # 최근 lookback_days 일 데이터를 사용하여 예측 시작
    last_observed = df_filtered[stock].iloc[-lookback_days:]

    # 데이터가 없는 구간에 대한 예측 수행
    future_pred_val = []
    remaining_days = (input_date - df_filtered.index[-1]).days

    while len(future_pred_val) < remaining_days:
        # 기존 모델 파라미터를 사용하여 예측
        pred = loaded_model.get_forecast(steps=steps_ahead)
        pred_mean = pred.predicted_mean.cumsum() + last_observed.iloc[-1]
        future_pred_val.extend(pred_mean)
        last_observed = pd.concat([last_observed, pred_mean]).iloc[-lookback_days:]

    # 예측된 날짜 생성
    future_pred_dates = pd.date_range(start=df_filtered.index[-1] + pd.Timedelta(days=1), 
                                      periods=len(future_pred_val), freq='D')

    # 예측된 데이터와 실제 데이터를 하나의 데이터프레임으로 결합
    pred_df = pd.DataFrame({'Predicted': future_pred_val[:remaining_days]}, index=future_pred_dates[:remaining_days])

    return print(f'{stock}', '예측 일 :', str(pred_df.index[-1])[:10],' 예측 값 :', pred_df[-1:].values[0][0])

# 함수 사용 예시
sarima_predict_up_to_date('sc_ss_stock', r'C:\Users\4호실-8\Desktop\Save_Models\Regression_Samsung_SARIMA.pkl')
sarima_predict_up_to_date('sc_ap_stock', r'C:\Users\4호실-8\Desktop\Save_Models\Regression_Apple_SARIMA.pkl')
sarima_predict_up_to_date('sc_coin', r'C:\Users\4호실-8\Desktop\Save_Models\Regression_Bitcoin_SARIMA.pkl')

sc_ss_stock 예측 일 : 2024-09-30  예측 값 : 63777.95204410909
sc_ap_stock 예측 일 : 2024-09-30  예측 값 : 227.40548896799604
sc_coin 예측 일 : 2024-09-30  예측 값 : 65314.04810709587


In [4]:
# 저장된 모델 파일 불러오기
model_path = r'C:\Users\4호실-8\Desktop\Save_Models\Regression_Samsung_Prophet_model.pkl'
with open(model_path, 'rb') as f:
    Regression_Samsung_Prophet = pickle.load(f)

model_path = r'C:\Users\4호실-8\Desktop\Save_Models\Regression_Apple_Prophet_model.pkl'
with open(model_path, 'rb') as f:
    Regression_Apple_Prophet = pickle.load(f)

model_path = r'C:\Users\4호실-8\Desktop\Save_Models\Regression_Bitcoin_Prophet_model.pkl'
with open(model_path, 'rb') as f:
    Regression_Bitcoin_Prophet = pickle.load(f)

def predict_next_year_from_date(start_date, model):
    """
    주어진 시작 날짜로부터 1년간의 주가 데이터를 예측하는 함수.
    
    :param model: 학습된 Prophet 모델
    :param start_date: 예측을 시작할 날짜 (YYYY-MM-DD 형식의 문자열)
    :return: 예측 결과 (DataFrame)
    """
    # 시작 날짜로부터 1년 후 날짜 계산
    future_dates = pd.date_range(start=start_date, periods=366, freq='D')
    future = pd.DataFrame({'ds': future_dates})
    
    # 예측 수행
    forecast = model.predict(future)

    # 예측된 주가 데이터 시각화
    # plt.figure(figsize=(10, 6))
    # plt.plot(forecast['ds'], forecast['yhat'], label='Predicted')
    # #plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='gray', alpha=0.2, label='Confidence Interval')
    # plt.title(f'Stock Price Prediction from {start_date} for 1 Year')
    # plt.xlabel('Date')
    # plt.ylabel('Predicted Price')
    # plt.legend()
    # plt.show()

    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

# 예시: 특정 날짜로부터 1년간의 예측
print(predict_next_year_from_date(current_date, Regression_Samsung_Prophet))
print(predict_next_year_from_date(current_date, Regression_Apple_Prophet))
print(predict_next_year_from_date(current_date, Regression_Bitcoin_Prophet))

            ds          yhat   yhat_lower     yhat_upper
361 2025-09-26  89190.740105 -2793.307395  182250.102059
362 2025-09-27  89323.348440 -2469.490350  184543.435823
363 2025-09-28  89443.155684 -2931.131334  184489.007172
364 2025-09-29  89548.910736 -4147.213597  183250.449420
365 2025-09-30  89639.746885 -3168.793425  184571.245410
            ds        yhat  yhat_lower  yhat_upper
361 2025-09-26  237.632154   77.638168  406.444211
362 2025-09-27  237.313440   78.476147  407.475002
363 2025-09-28  237.054268   75.852452  409.003914
364 2025-09-29  236.850097   77.290068  408.376119
365 2025-09-30  236.695773   76.182285  408.913258
            ds          yhat    yhat_lower     yhat_upper
361 2025-09-26  68338.145632  14462.283237  120020.722008
362 2025-09-27  68260.261131  13696.893710  119734.444185
363 2025-09-28  68276.371200  13518.391215  119994.720494
364 2025-09-29  68387.150915  12620.945984  119059.032797
365 2025-09-30  68591.942230  13211.091146  121167.824971


# XGBoost

In [5]:
def xgboost_predict(stock, df, day, model_filename):
    # 데이터 전처리
    stock_dataset = df[['sc_date', stock]]
    stock_dataset['sc_date'] = pd.to_datetime(stock_dataset['sc_date'])
    stock_dataset.rename(columns={'sc_date': 'Date', stock: 'stock'}, inplace=True)
    stock_dataset.set_index('Date', inplace=True)
    
    # 날짜 변환 및 계산
    today = datetime.today()
    month_format = str(today.month).zfill(2)  # 월 형식 정리
    day_format = str(today.day).zfill(2)      # 일 형식 정리
    today_format = datetime.strptime(f'{today.year}{month_format}{day_format}', '%Y%m%d')
    formatted_date = datetime.strptime(str(day), '%Y%m%d')

    start_day = 4  # 모델이 학습한 특성 수 (예시로 4로 설정)
    end_day = (formatted_date - today_format).days + 1

    # 날짜가 오늘보다 이전일 경우 오류 처리
    if end_day < 1:
        print("예측 날짜는 오늘 이후여야 합니다. 올바른 날짜를 입력하세요.")
        return

    # 데이터 준비
    days_in = start_day  # 모델이 기대하는 입력 데이터 특성 수로 조정
    day_out = 1          # 예측할 일 수
    raw = []

    # 특성 생성: 과거 days_in일과 미래 day_out일을 기준으로 이동
    for i in range(days_in, 0, -1):
        raw.append(stock_dataset.shift(i))
    for i in range(0, day_out):
        raw.append(stock_dataset.shift(-i))

    sum_df = pd.concat(raw, axis=1)
    sum_df.dropna(inplace=True)

    # 데이터 분리
    train = sum_df.values
    X = train[:, :-1]
    Y = train[:, -1]

    # 저장된 모델 로드
    model = joblib.load(model_filename)
    print(f'Model loaded from {model_filename}')

    # 예측 데이터 준비
    data_in = stock_dataset[-(days_in):]  # 예측을 위한 마지막 days_in일 데이터 슬라이싱
    data_in = data_in.values[-4:]         # 마지막 4일치 데이터만 사용 (모델의 기대 특성 수와 일치하도록)
    data_in = data_in.flatten().reshape(1, -1)  # (1, n_features) 형태로 변환

    forecast_days = end_day  # 예측할 전체 값
    forecast_results = []

    # 반복하여 예측 수행
    for _ in range(forecast_days):
        # 예측 수행
        prediction = model.predict(data_in)
        forecast_results.append(prediction[0])

        # 데이터 업데이트: 예측 값을 데이터에 추가하고 가장 오래된 데이터를 제거
        new_data = np.append(data_in[0][1:], prediction)
        data_in = new_data.reshape(1, -1)

    # 예측 결과를 DataFrame으로 변환
    forecast_dates = pd.date_range(start=stock_dataset.index[-1], periods=forecast_days)
    forecast_df = pd.DataFrame(forecast_results, index=forecast_dates, columns=['Forecast'])

    # 예측 결과 출력
    print('예측 일:', str(forecast_df.index[-1])[:10], ' 예측 값:', forecast_df.iloc[-1].values[0])

# 사용 예시:
# 예측할 날짜를 지정하고, 모델 파일 경로를 설정합니다.
current_date1 = int(str(current_date).replace('-', ''))
xgboost_predict('sc_ss_stock', df, current_date1, model_filename=r'C:\Users\4호실-8\Desktop\Save_Models\regression_samsung_xgboost.pkl')
xgboost_predict('sc_ap_stock', df, current_date1, model_filename=r'C:\Users\4호실-8\Desktop\Save_Models\regression_apple_xgboost.pkl')
xgboost_predict('sc_coin', df, current_date1, model_filename=r'C:\Users\4호실-8\Desktop\Save_Models\regression_bitcoin_xgboost.pkl')

Model loaded from C:\Users\4호실-8\Desktop\Save_Models\regression_samsung_xgboost.pkl
예측 일: 2024-09-25  예측 값: 63398.33
Model loaded from C:\Users\4호실-8\Desktop\Save_Models\regression_apple_xgboost.pkl
예측 일: 2024-09-25  예측 값: 228.86752
Model loaded from C:\Users\4호실-8\Desktop\Save_Models\regression_bitcoin_xgboost.pkl
예측 일: 2024-09-25  예측 값: 63250.656


In [6]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # TensorFlow 로그 숨김
tf.get_logger().setLevel('ERROR')  # absl 로그 숨기기

# Keras 모델을 불러오는 함수 정의
def load_keras_model(model_path):
    # Keras 모델 로드
    model = keras_load_model(model_path)
    return model

# 주가를 예측하는 함수 정의
def load_model_and_predict(stock, model_path, data_path='../../data/tb_stock.xlsx', look_back=10, days_ahead=30):
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    # 모델 로드
    best_model = load_keras_model(model_path)

    # 데이터 로드 및 전처리
    df = pd.read_excel(data_path)
    df = df[['sc_date', stock]]
    df = df[:-30]
    df['sc_date'] = pd.to_datetime(df['sc_date'])  # datetime 형식으로 변환
    df.dropna(inplace=True)  # 결측치 제거

    scaled_data = scaler.fit_transform(df[[stock]])

    # 미래 예측 함수 정의
    def predict_future(stock, data, model, scaler, look_back=look_back, days_ahead=days_ahead):
        future_predictions = []
        last_sequence = data[-look_back:]  # 가장 최근의 데이터 시퀀스

        for _ in range(days_ahead):
            pred = model.predict(last_sequence.reshape(1, look_back, 1), verbose=0)  # verbose=0으로 출력 숨기기
            future_predictions.append(pred[0, 0])
            last_sequence = np.append(last_sequence[1:], pred, axis=0)

        result = np.array(future_predictions)

        predicted_price_30_days = result[-1]
        predicted_price_actual = scaler.inverse_transform([[predicted_price_30_days]])[0, 0]

        return predicted_price_actual  # 실제 가격 반환

    # 예측 수행
    predicted_price = predict_future(stock, scaled_data, best_model, scaler)
    print(f"{stock} 예측 가격 (30일 후): {predicted_price}")

# 모델 파일 경로
a = r'C:\Users\4호실-8\Desktop\Save_Models\regression_apple_lstm.h5'
b = r'C:\Users\4호실-8\Desktop\Save_Models\regression_samsung_lstm.h5'
c = r'C:\Users\4호실-8\Desktop\Save_Models\regression_bitcoin_lstm.h5'

# 예측 실행
load_model_and_predict('sc_ss_stock', b)
load_model_and_predict('sc_ap_stock', a)
load_model_and_predict('sc_coin', c)

sc_ss_stock 예측 가격 (30일 후): 72301.38832865877


sc_ap_stock 예측 가격 (30일 후): 185.13260169614185
sc_coin 예측 가격 (30일 후): 57588.84086937428


In [7]:

# 모델을 불러오는 함수 정의
def load_model(model_name):
    file_path = fr'C:\Users\4호실-8\Desktop\Save_Models\Regression_{model_name}_ExponentialSmoothing.pkl'
    with open(file_path, 'rb') as file:
        model = pickle.load(file)
    return model

# 주가를 예측하는 함수 정의
def predict_stock(stock, model, date=current_date):
    # 예측하고자 하는 날짜를 datetime 객체로 변환
    if isinstance(date, datetime):
        predict_date = date
    elif isinstance(date, str):
        predict_date = datetime.strptime(date, '%Y-%m-%d')
    elif isinstance(date, date):
        predict_date = datetime.combine(date, datetime.min.time())
    else:
        raise TypeError("date는 문자열, datetime.date 또는 datetime.datetime이어야 합니다.")
    
    # 모델의 마지막 훈련 날짜를 가져옴
    train_last_date = model.fittedvalues.index[-1]
    
    # train_last_date가 datetime.date 또는 datetime.datetime 객체일 경우 적절히 변환
    if isinstance(train_last_date, str):
        train_last_date = datetime.strptime(train_last_date, '%Y-%m-%d')
    elif isinstance(train_last_date, date) and not isinstance(train_last_date, datetime):
        train_last_date = datetime.combine(train_last_date, datetime.min.time())
    elif isinstance(train_last_date, datetime):
        train_last_date = train_last_date
    else:
        raise TypeError("train_last_date는 문자열, datetime.date 또는 datetime.datetime이어야 합니다.")
    
    # 예측하고자 하는 날짜와 훈련 마지막 날짜의 차이 계산
    date_sub = (predict_date - train_last_date).days
    
    # 예측할 일수가 양수인지 확인 (음수면 과거 예측이라 에러)
    if date_sub <= 0:
        raise ValueError("예측 날짜는 모델 학습 마지막 날짜 이후여야 합니다.")
    
    # 주어진 날짜까지 예측 수행
    predictions = model.forecast(steps=date_sub)
    
    # 마지막 예측 값 반환
    return f'{stock} {predict_date.date()}: {predictions[-1]}'

# 모델 불러오기
samsung_model = load_model('Samsung')
apple_model = load_model('Apple')
coin_model = load_model('Coin')

# 예시 사용: 날짜를 문자열로 명시하여 전달합니다.
current_date = str(current_date)
print(predict_stock('Samsung', samsung_model, current_date))
print(predict_stock('Apple', apple_model, current_date))
print(predict_stock('Coin', coin_model, current_date))

Samsung 2024-09-30: 64729.98752316575
Apple 2024-09-30: 318.56251440626505
Coin 2024-09-30: 45160.50298790372
